In [1]:
# Build a toy example: sequence of numbers as input → reverse of that sequence as output.

In [2]:
#pip install tensorflow

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [9]:
# Sample config
num_samples = 2000
max_seq_len = 10
num_features = 10  # e.g., 0-9 one-hot encoding

In [10]:
# Generate dummy data
def generate_data(n_samples, seq_len, vocab_size):
    X = np.random.randint(1, vocab_size, (n_samples, seq_len))
    y = np.flip(X, axis=1)  # Reverse of input sequence
    return X, y

X_raw, y_raw = generate_data(num_samples, max_seq_len, num_features)

In [11]:
X_raw[0], y_raw[0]

(array([8, 9, 5, 4, 8, 9, 9, 5, 4, 1]), array([1, 4, 5, 9, 9, 8, 4, 5, 9, 8]))

In [12]:
# One-hot encoding input and output
def one_hot_encode(seqs, num_classes):
    return tf.keras.utils.to_categorical(seqs, num_classes=num_classes)

encoder_input_data = one_hot_encode(X_raw, num_features)
decoder_target_data = one_hot_encode(y_raw, num_features)

In [13]:
# Shift decoder input for Teacher Forcing (target shifted by one step)
decoder_input_data = np.zeros_like(decoder_target_data)
decoder_input_data[:, 1:, :] = decoder_target_data[:, :-1, :]
decoder_input_data[:, 0, 0] = 1  # Start token as index 0

In [14]:
# ----------- ENCODER ----------------
encoder_inputs = Input(shape=(None, num_features), name='encoder_inputs')
encoder_lstm = LSTM(64, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]  # Pass these to decoder

In [15]:
# The final output goes through a Dense layer with softmax to get probabilities.
# ----------- DECODER ----------------
decoder_inputs = Input(shape=(None, num_features), name='decoder_inputs')
decoder_lstm = LSTM(64, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_features, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

In [16]:
# ----------- MODEL ------------------
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None, 10)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm (LSTM) │ [(None, 64),      │     19,200 │ encoder_inputs[0… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm (LSTM) │ [(None, None,     │     19,200 │ decoder_inputs[0… │
│                     │ 64), (None, 64),  │            │ encoder_lstm[0][… │
│                     │ (None, 64)]       │            │ encoder_lstm[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense       │ (None, None, 10)  │        650 │ decoder_lstm[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,050 (152.54 KB)

 Trainable params: 39,050 (152.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
"""
Encoder =>
Takes the input sequence (e.g., 1, 5, 9, 2) and compresses it into a fixed-size context vector.

This vector holds the “summary” of the input sentence.

We use LSTM here so it can remember long-term patterns.

Decoder =>

Starts with the context vector and tries to generate the output sequence one step at a time.

It uses teacher forcing during training (real previous token given as input).

Uses LSTM again for generation."""

'\nEncoder =>\nTakes the input sequence (e.g., 1, 5, 9, 2) and compresses it into a fixed-size context vector.\n\nThis vector holds the “summary” of the input sentence.\n\nWe use LSTM here so it can remember long-term patterns.\n\nDecoder =>\n\nStarts with the context vector and tries to generate the output sequence one step at a time.\n\nIt uses teacher forcing during training (real previous token given as input).\n\nUses LSTM again for generation.'

In [17]:
# ----------- TRAIN ------------------
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,
    epochs=20,
    validation_split=0.2
)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.1357 - loss: 2.2594 - val_accuracy: 0.1990 - val_loss: 2.1065
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2435 - loss: 2.0341 - val_accuracy: 0.2920 - val_loss: 1.8572
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3179 - loss: 1.8079 - val_accuracy: 0.3548 - val_loss: 1.7100
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3775 - loss: 1.6458 - val_accuracy: 0.4315 - val_loss: 1.5209
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4573 - loss: 1.4496 - val_accuracy: 0.5032 - val_loss: 1.3468
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5469 - loss: 1.2695 - val_accuracy: 0.5995 - val_loss: 1.1558
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6092 - loss: 1.1127 - val_accuracy: 0.6430 - val_loss: 1.0272
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6734 - loss: 0.9729 - val_accuracy: 0.6320 - val_l